In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from nuztf.neutrino_scanner import NeutrinoScanner
from astropy import units as u
from nuwinter.avro import load_avro
from nuwinter.plot import ann_fields, generate_single_page
from nuwinter.utils import deduplicate_df
from nuwinter.paths import get_pdf_path
from pathlib import Path
import pandas as pd
import json
from astropy import units as u
from astropy.coordinates import SkyCoord
from matplotlib.colors import Normalize
from matplotlib.ticker import MultipleLocator
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
from tqdm import tqdm
from nuztf.skymap_scanner import SkymapScanner







/Users/robertstein/anaconda3/envs/nuwinter_env/lib/python3.11/site-packages/ligo/lw/lsctables.py:89: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  import lal


In [2]:
base_dir = Path.home().joinpath("Data/winter")

name = "IC240412A"
name = "S240422ed"

prob_threshold = 0.90
n_days = 7

In [3]:
outpath = str(get_pdf_path(name)).replace("combined", "combined_ztf")
outpath

'/Users/robertstein/Data/nuwinter/S240422ed/winter_candidate_pdf/combined_ztf.pdf'

In [4]:
nu = NeutrinoScanner(name) if "IC" in name else SkymapScanner(
    event=name,
    prob_threshold=prob_threshold,
    n_days=n_days
)

100%|█████████████████████████████████████| 506/506 [00:00<00:00, 249743.21it/s]


In [7]:
ztf_initial_cache = nu.get_initial_cache_path()

download_ztf = False

if not (ztf_initial_cache.exists()):
    download_ztf = True

if ztf_initial_cache.exists():
    if not ztf_initial_cache.stat().st_size > 2:
        download_ztf = True

if download_ztf:
    print(f"No file found at {ztf_initial_cache}")
    if isinstance(nu, SkymapScanner):
        nu.get_alerts()
        nu.filter_alerts()
    else:
        nu.scan_area(t_max=nu.t_min + 14.)

ztf_initial_cache

No file found at /Users/robertstein/Data/nuztf/cache/candidates/S240422ed/rev3/0.9/initial_stage.json


0it [00:00, ?it/s]
0it [00:00, ?it/s]


PosixPath('/Users/robertstein/Data/nuztf/cache/candidates/S240422ed/rev3/0.9/initial_stage.json')

In [8]:
nights = [str(nu.t_min + (i - 1) * u.day).split("T")[0].replace("-", "") for i in range(n_days + 1)]

all_res = []

for night in sorted(nights):
    
    avro_dir = base_dir / f"{night}/avro"

    print(f"Looking for avros in {avro_dir}")
    
    avro_files = list(avro_dir.glob("*.avro"))
    
    for path in avro_files:
        all_res.append(load_avro(path))

if len(all_res) == 0:
    raise FileNotFoundError(f"No avro files found for any night, please download these first using the download notebook!")

res_winter = pd.concat(all_res)

Looking for avros in /Users/robertstein/Data/winter/20240421/avro
Looking for avros in /Users/robertstein/Data/winter/20240422/avro
Looking for avros in /Users/robertstein/Data/winter/20240423/avro
Looking for avros in /Users/robertstein/Data/winter/20240424/avro
Looking for avros in /Users/robertstein/Data/winter/20240425/avro
Looking for avros in /Users/robertstein/Data/winter/20240426/avro
Looking for avros in /Users/robertstein/Data/winter/20240427/avro
Looking for avros in /Users/robertstein/Data/winter/20240428/avro


FileNotFoundError: No avro files found for any night, please download these first using the download notebook!

In [9]:
in_contour = []

for _, row in res_winter.iterrows():
    in_contour.append(nu.in_contour(row["ra"], row["dec"]))

print(f"{np.sum(in_contour)} / {len(in_contour)} candidates in contour")

res_winter = res_winter[in_contour]

NameError: name 'res_winter' is not defined

In [ ]:
res_winter

In [ ]:
with open(ztf_initial_cache, "r") as f:
    res_ztf = json.load(f)
    
ztf_sources = []

for entry in res_ztf:

    new_dict = {}
    
    for field in ["candid", "objectId"]:
        new_dict[field] = entry[field]

    for key, val in entry["candidate"].items():
        new_dict[key] = val

    ztf_sources.append(new_dict)

ztf_df = pd.DataFrame(ztf_sources)
ztf_df

In [ ]:
ztf_df

In [ ]:
if len(ztf_df) > 0:
    ztf_positions = SkyCoord(ztf_df["ra"].to_numpy()*u.deg, ztf_df["dec"].to_numpy()*u.deg, frame='icrs')
    
    winter_positions = SkyCoord(res_winter["ra"].to_numpy()*u.deg, res_winter["dec"].to_numpy()*u.deg, frame='icrs')
    idx, d2d, _ = winter_positions.match_to_catalog_sky(ztf_positions)
    
    crossmatch_radius = 3.0 * u.arcsec
    mask = d2d < crossmatch_radius
    
    cut = res_winter[mask]
    cut.reset_index(inplace=True)
    cut

In [ ]:
if len(ztf_df) > 0:
    ztf_names = []

    for i in idx[mask]:
        ztf_names.append(ztf_df["objectId"].iloc[i])
    
    cut = cut.assign(ztf_name=ztf_names)
    if len(cut) > 0:
        cut = deduplicate_df(cut)
    cut
else:
    cut = pd.DataFrame()

In [ ]:
if len(cut) > 0:
    outpath = str(get_pdf_path(name)).replace("combined", "combined_ztf")
    
    print(f"Saving PDF to {outpath}")
    
    with PdfPages(outpath) as pdf:
        for i, row in tqdm(cut.iterrows(), total=len(cut)):
            generate_single_page(row, ann_fields=["ztf_name"] + ann_fields)
            pdf.savefig()
            plt.close()
else:
    print("No candidates found")

In [ ]:
len(res_winter["objectid"])

In [ ]:
crossmatch_radius = 2.0 * u.arcsec

winter_positions = SkyCoord(res_winter["ra"].to_numpy()*u.deg, res_winter["dec"].to_numpy()*u.deg, frame='icrs')


match = []

for i, c in enumerate(winter_positions):
    if i < len(winter_positions) - 1:
        idx, d2d, _ = c.match_to_catalog_sky(winter_positions[i+1:])
        if d2d < crossmatch_radius:
            match.append(idx)
        else:
            match.append(None)
    else:
        match.append(None)

In [ ]:
res_winter["match"] = match
res_winter_match = res_winter[pd.notnull(match)]
res_winter_match

In [ ]:
all_matched = []

for i, row in res_winter_match.iterrows():
    c = SkyCoord(row["ra"]*u.deg, row["dec"]*u.deg, frame='icrs')
    sep = c.separation(winter_positions)
    mask = (sep.arcsec > 0.) & (sep.arcsec < crossmatch_radius.value)
    matches = res_winter.loc[mask]
    row["ndethist"] = mask.sum()
    row["prv_candidates"] = res_winter.loc[mask]
    print(row["objectid"], res_winter.loc[mask]["objectid"])
    all_matched.append(row)

res_winter_match = pd.concat(all_matched, axis=1).transpose()
res_winter_match

In [ ]:
res_winter_match["ndethist"]

In [ ]:
sgscore_cut = 0.5
dist_cut_arcsec = 5.

star_cut = (pd.notnull(res_winter_match["distpsnr1"]) & (res_winter_match["distpsnr1"] < dist_cut_arcsec) & pd.notnull(res_winter_match["sgscore1"]) & (res_winter_match["sgscore1"] > sgscore_cut))

masks = [
    res_winter_match["ndethist"] > 0,
    ~star_cut
]

mask = np.ones(len(res_winter_match), dtype=bool)
for new_mask in masks:
    mask *= new_mask
    
print(f"{np.sum(mask)} / {len(mask)} detections pass all cuts")
cut2 = res_winter_match[mask]

if len(cut2) > 0:
    cut2 = deduplicate_df(cut2)
    cut2.sort_values(by="objectid", inplace=True)
    
    outpath = str(get_pdf_path(name)).replace("combined", "combined_2dets")
    print(f"Saving PDF to {outpath}")
    
    with PdfPages(outpath) as pdf:
        for i, row in tqdm(cut2.iterrows(), total=len(cut2)):
            generate_single_page(row, ann_fields=ann_fields)
            pdf.savefig()
            plt.close()

In [ ]:
list(res_winter_match.columns)